# Traj-XAI Final Experiement (Colab version)

This notebook runs through multiple experiments and calculates their fidelity metrics.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install required packages if needed
!pip install rdp fastdtw pactus

In [ ]:
# form a symbolic link to the codebase found in the mounted Google Drive
!ln -s "/content/drive/MyDrive/XAI4Traj/traj_xai" "/content/traj_xai"

In [ ]:
import sys
sys.path.append('/content/traj_xai')

In [ ]:
from pactus import Dataset
from pactus.models import LSTMModel

# import every model, segmementation method, and perturbation method
from traj_xai.src import (
    SimpleGRUModel, SimpleTransformerModel, TrajFormerModel, 
    sliding_window_segmentation, mdl_segmentation, rdp_segmentation, 
    gaussian_perturbation, scaling_perturbation, rotation_perturbation, gan_perturbation, 
    run_experiments, 
)

# Load Dataset(s)


In [ ]:
SEED = 0

# datasets = [
#     # Dataset.geolife(),
#     Dataset.animals(),
#     Dataset.hurdat2(),
#     Dataset.uci_characters(),
#     Dataset.uci_pen_digits(),
#     Dataset.uci_movement_libras(),
#     Dataset.mnist_stroke()
# ]

# train_sets = []
# test_sets = []
# for ds in datasets:
#     train, test = ds.split(0.8, random_state=SEED)
#     train_sets.append(train)
#     test_sets.append(test)

# Load dataset
dataset = Dataset.uci_characters()

# Split data into train and test subsets
train, test = dataset.split(0.8, random_state=SEED)

## ML Model (unused currently)


In [5]:
# # ML Model
# SEED = 0

# # datasets = [
# #     # Dataset.geolife(),
# #     Dataset.animals(),
# #     # Dataset.hurdat2(), -> RF cant work
# #     Dataset.cma_bst(),
# #     # Dataset.mnist_stroke(), -> okay
# #     # Dataset.uci_pen_digits(),
# #     # Dataset.uci_gotrack(),
# #     Dataset.uci_characters(),
# #     Dataset.uci_movement_libras(),-> okay
# # ]

# # Load dataset
# dataset = Dataset.geolife()

# # Split data into train and test subsets
# # train, test = dataset.split(.995, random_state=SEED)

# train, test = dataset.filter(
#         lambda traj, label: len(traj) >= 5
#         and traj.r.delta.norm.sum() > 0
#         and dataset.label_counts[label] > 5
#     ).split(
#         train_size=0.985,
#         random_state=SEED,
#     )

# # Build and train the model
# featurizer = featurizers.UniversalFeaturizer()
# model = XGBoostModel(featurizer=featurizer)
# model = SVMModel(
#     featurizer=featurizer,
#     random_state=SEED,
# )
# DecisionTreeModel(
#     featurizer=featurizer,
#     max_depth=7,
#     random_state=SEED,
# )


# # Train the model on the train dataset
# model.train(train, 5) # 5: CV

# # Evaluate the model on a test dataset
# evaluation = model.evaluate(test)

# # Print the evaluation
# evaluation.show()

## DL Model


In [ ]:
# Build and train the model
model = TrajFormerModel(c_out=len(dataset.classes), random_state=SEED)

# Train the model on the train dataset
model.train(train, dataset, epochs=10, batch_size=64)

# Evaluate the model on a test dataset
evaluation = model.evaluate(test)

# Print the evaluation
evaluation.show()

23:03:30 [INFO] Epoch 10/10, Loss: 28.1342
23:03:30 [INFO] Trained TrajFormer model with 2286 samples



General statistics:

Accuracy: 0.724
F1-score: 0.705
Mean precision: 0.750
Mean recall: 0.710

Confusion matrix:

a      b      c      d      e      g      h      l      m      n      o      p      q      r      s      u      v      w      y      z      precision
44.12  3.57   0.0    0.0    0.0    7.14   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    3.85   0.0    0.0    0.0    0.0    78.95  
8.82   85.71  0.0    0.0    0.0    3.57   0.0    0.0    0.0    0.0    0.0    0.0    8.0    4.17   0.0    26.92  0.0    4.0    3.7    0.0    60.0   
0.0    0.0    93.1   0.0    0.0    3.57   0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    96.43  
0.0    0.0    0.0    90.62  0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    100.0  
0.0    0.0    0.0    0.0    94.59  0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    100

# Experiment


In [ ]:
import logging

logging.getLogger().setLevel(logging.INFO)

In [6]:
segment_func = [sliding_window_segmentation]
perturbation_func = [gaussian_perturbation]

run_experiments(test, segment_func, perturbation_func, model)

23:04:19 [INFO] Running experiment with sliding_window_segmentation and gaussian_perturbation
23:04:19 [DEBUG] Starting explanation process...
23:04:20 [DEBUG] Normalizing predictions: type=<class 'numpy.ndarray'>, shape=(1024, 20)
23:05:04 [INFO] Finished explanation process.
23:05:04 [DEBUG] Starting get_Y process...
23:05:04 [DEBUG] Normalizing predictions: type=<class 'numpy.ndarray'>, shape=(1, 20)
23:05:04 [DEBUG] Starting get_Y_eval_sorted process...
23:05:05 [DEBUG] Normalizing predictions: type=<class 'numpy.ndarray'>, shape=(38, 20)
23:05:05 [DEBUG] Starting get_Y process...
23:05:05 [DEBUG] Normalizing predictions: type=<class 'numpy.ndarray'>, shape=(1, 20)
23:05:05 [DEBUG] Finished get_Y_eval_sorted process.
23:05:05 [DEBUG] Starting explanation process...


[INFO] Row saved to logs\sliding_window_segmentation_gaussian_perturbation_results.csv: [0, 'traj_b477410c', 0, 0.0, 'ok']
2025-11-27 23:05:05 - Total RAM: 13.86 GB | Used: 82.7%


23:05:06 [DEBUG] Normalizing predictions: type=<class 'numpy.ndarray'>, shape=(1024, 20)
23:05:06 [WARNING] Only one class detected, skipping explanation.
23:05:06 [INFO] Trajectory 1: classification unchanged
23:05:06 [DEBUG] Starting explanation process...


[INFO] Row saved to logs\sliding_window_segmentation_gaussian_perturbation_results.csv: [1, 'traj_9b6e1be1', 0, 0.0, 'ok']


23:05:07 [DEBUG] Normalizing predictions: type=<class 'numpy.ndarray'>, shape=(1024, 20)
23:05:51 [INFO] Finished explanation process.
23:05:51 [DEBUG] Starting get_Y process...
23:05:51 [DEBUG] Normalizing predictions: type=<class 'numpy.ndarray'>, shape=(1, 20)
23:05:51 [DEBUG] Starting get_Y_eval_sorted process...
23:05:51 [DEBUG] Normalizing predictions: type=<class 'numpy.ndarray'>, shape=(38, 20)
23:05:51 [DEBUG] Starting get_Y process...
23:05:51 [DEBUG] Normalizing predictions: type=<class 'numpy.ndarray'>, shape=(1, 20)
23:05:51 [DEBUG] Finished get_Y_eval_sorted process.
23:05:51 [DEBUG] Starting explanation process...


...

[INFO] Row saved to logs\sliding_window_segmentation_gaussian_perturbation_results.csv: [569, 'traj_7cefd64b', 0, 0.0, 'ok']


04:24:22 [DEBUG] Normalizing predictions: type=<class 'numpy.ndarray'>, shape=(1024, 20)
04:24:59 [INFO] Finished explanation process.
04:24:59 [DEBUG] Starting get_Y process...
04:24:59 [DEBUG] Normalizing predictions: type=<class 'numpy.ndarray'>, shape=(1, 20)
04:24:59 [DEBUG] Starting get_Y_eval_sorted process...
04:24:59 [DEBUG] Normalizing predictions: type=<class 'numpy.ndarray'>, shape=(35, 20)
04:24:59 [DEBUG] Starting get_Y process...
04:24:59 [DEBUG] Normalizing predictions: type=<class 'numpy.ndarray'>, shape=(1, 20)
04:24:59 [DEBUG] Finished get_Y_eval_sorted process.
04:24:59 [DEBUG] Starting explanation process...


[INFO] Row saved to logs\sliding_window_segmentation_gaussian_perturbation_results.csv: [570, 'traj_36eb6fe3', 0, 0.0, 'ok']
2025-11-28 04:24:59 - Total RAM: 13.86 GB | Used: 56.7%


04:25:00 [DEBUG] Normalizing predictions: type=<class 'numpy.ndarray'>, shape=(1024, 20)
04:25:00 [WARNING] Only one class detected, skipping explanation.
04:25:00 [INFO] Trajectory 571: classification unchanged
04:25:00 [INFO] Results saved to logs\sliding_window_segmentation_gaussian_perturbation_results.csv


[INFO] Row saved to logs\sliding_window_segmentation_gaussian_perturbation_results.csv: [571, 'traj_1ae9a8cc', 0, 0.0, 'ok']


In [10]:
count = 0
total_change = 0
avg_precision = 0
with open("logs/sliding_window_segmentation_gaussian_perturbation_results.csv") as f:
    for i, line in enumerate(f):
        if i > 0 and len(line) > 0:
            count += 1
            _, _, change, precision, _ = line.split(',')
            total_change += int(change)
            avg_precision += float(precision)
recall = total_change / count
avg_precision /= total_change
h_mean = (2 * avg_precision * recall) / (avg_precision + recall)
print(f"Recall: {recall:.3f}\nPrecision: {avg_precision:.3f}\nF1 Score: {h_mean:.3f}")

Recall: 0.449
Precision: 1.000
F1 Score: 0.620
